In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame,Series
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

Congratulations – you have been hired as Chief Data Scientist of MedCamp – a not for profit organization dedicated in making health conditions for working professionals better. MedCamp was started because the founders saw their family suffer due to bad work life balance and neglected health.

MedCamp organizes health camps in several cities with low work life balance. They reach out to working people and ask them to register for these health camps. For those who attend, MedCamp provides them facility to undergo health checks or increase awareness by visiting various stalls (depending on the format of camp). 

MedCamp has conducted 65 such events over a period of 4 years and they see a high drop off between “Registration” and Number of people taking tests at the Camps. In last 4 years, they have stored data of ~110,000 registrations they have done.

One of the huge costs in arranging these camps is the amount of inventory you need to carry. If you carry more than required inventory, you incur unnecessarily high costs. On the other hand, if you carry less than required inventory for conducting these medical checks, people end up having bad experience.

 

The Process:
MedCamp employees / volunteers reach out to people and drive registrations.
During the camp, People who “ShowUp” either undergo the medical tests or visit stalls depending on the format of health camp.
 

Other things to note:
Since this is a completely voluntary activity for the working professionals, MedCamp usually has little profile information about these people.
For a few camps, there was hardware failure, so some information about date and time of registration is lost.
MedCamp runs 3 formats of these camps. The first and second format provides people with an instantaneous health score. The third format provides information about several health issues through various awareness stalls.
Favorable outcome:
For the first 2 formats, a favourable outcome is defined as getting a health_score, while in the third format it is defined as visiting at least a stall.
You need to predict the chances (probability) of having a favourable outcome.
 

Data Description
Train.zip contains the following 6 csvs alongside the data dictionary that contains definitions for each variable

Health_Camp_Detail.csv – File containing Health_Camp_Id, Camp_Start_Date, Camp_End_Date and Category details of each camp.

Train.csv – File containing registration details for all the test camps. This includes Patient_ID, Health_Camp_ID, Registration_Date and a few anonymized variables as on registration date.

Patient_Profile.csv – This file contains Patient profile details like Patient_ID, Online_Follower, Social media details, Income, Education, Age, First_Interaction_Date, City_Type and Employer_Category

First_Health_Camp_Attended.csv – This file contains details about people who attended health camp of first format. This includes Donation (amount) & Health_Score of the person.

Second_Health_Camp_Attended.csv - This file contains details about people who attended health camp of second format. This includes Health_Score of the person.

Third_Health_Camp_Attended.csv - This file contains details about people who attended health camp of third format. This includes Number_of_stall_visited & Last_Stall_Visited_Number.

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/Train.csv')
pp=pd.read_csv('/content/drive/MyDrive/Patient_Profile.csv')
hcdetail=pd.read_csv('/content/drive/MyDrive/Health_Camp_Detail.csv')
fhc=pd.read_csv('/content/drive/MyDrive/First_Health_Camp_Attended.csv')
shc=pd.read_csv('/content/drive/MyDrive/Second_Health_Camp_Attended.csv')
thc=pd.read_csv('/content/drive/MyDrive/Third_Health_Camp_Attended.csv')
test=pd.read_csv('/content/drive/MyDrive/test_l0Auv8Q.csv')

In [ ]:
train.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,523729,6534,29-Apr-06,0,0,0,0,0
3,524931,6535,07-Feb-04,0,0,0,0,0
4,521364,6529,28-Feb-06,15,1,0,0,7


In [ ]:
pp.columns

Index(['Patient_ID', 'Online_Follower', 'LinkedIn_Shared', 'Twitter_Shared',
       'Facebook_Shared', 'Income', 'Education_Score', 'Age',
       'First_Interaction', 'City_Type', 'Employer_Category'],
      dtype='object')

In [ ]:
hcdetail.columns

Index(['Health_Camp_ID', 'Camp_Start_Date', 'Camp_End_Date', 'Category1',
       'Category2', 'Category3'],
      dtype='object')

In [ ]:
combined=pd.concat([train,test],ignore_index=True)

In [ ]:
# Link all the additional Data Info available in different dfs...
combined = pd.merge(combined, pp, on = ["Patient_ID"], how = "left")

# First Health Camp
combined = pd.merge(combined, fhc, on = ["Patient_ID","Health_Camp_ID"], 
                    how = "left")

# Second Health Camp
combined = pd.merge(combined, shc, 
                    on = ["Patient_ID","Health_Camp_ID"], how = "left")

# Second Health Camp
combined = pd.merge(combined, thc, 
                    on = ["Patient_ID","Health_Camp_ID"], how = "left")

combined = pd.merge(combined, hcdetail, on = ["Health_Camp_ID"], how = "left")



In [ ]:
combined.shape

(110527, 29)

In [ ]:
pd.options.display.max_columns=None

In [ ]:
combined.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Donation,Health_Score,Unnamed: 4,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,489652,6578,10-Sep-05,4,0,0,0,2,0,0,0,0,None,None,None,06-Dec-04,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,16-Aug-05,14-Oct-05,Third,G,2
1,507246,6578,18-Aug-05,45,5,0,0,7,0,0,0,0,1,75,40,08-Sep-04,C,Others,NaN,NaN,NaN,NaN,NaN,NaN,16-Aug-05,14-Oct-05,Third,G,2
2,523729,6534,29-Apr-06,0,0,0,0,0,0,0,0,0,None,None,None,22-Jun-04,NaN,NaN,NaN,NaN,NaN,0.402054,NaN,NaN,17-Oct-05,07-Nov-07,Second,A,2
3,524931,6535,07-Feb-04,0,0,0,0,0,0,0,0,0,None,None,None,07-Feb-04,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01-Feb-04,18-Feb-04,First,E,2
4,521364,6529,28-Feb-06,15,1,0,0,7,0,0,0,1,1,70,40,04-Jul-03,I,Technology,NaN,NaN,NaN,0.845597,NaN,NaN,30-Mar-06,03-Apr-06,Second,A,2


In [ ]:
# social media
combined['social_media']=combined['Online_Follower']+combined['LinkedIn_Shared']+combined['LinkedIn_Shared']+combined['Twitter_Shared']+combined['Facebook_Shared']

In [ ]:
combined.social_media.value_counts()

0    98940
3     3391
2     3390
4     1934
1     1840
5     1032
Name: social_media, dtype: int64

In [ ]:
combined["Registration_Date"] = pd.to_datetime(combined.Registration_Date, 
                                               dayfirst=True)# Convert the data in Datetime Format
combined["Camp_Start_Date"] = pd.to_datetime(combined.Camp_Start_Date, 
                                             dayfirst=True)
combined["Camp_End_Date"] = pd.to_datetime(combined.Camp_End_Date, 
                                           dayfirst=True)
combined["First_Interaction"] = pd.to_datetime(combined.First_Interaction, 
                                               dayfirst=True)


In [ ]:
combined['days_interaction']=abs(combined.Registration_Date-combined.First_Interaction).dt.days

In [ ]:
combined['camp_duration']=abs(combined.Camp_Start_Date-combined.Camp_End_Date).dt.days

In [ ]:
combined['camp_registration']=abs(combined.Registration_Date-combined.Camp_Start_Date).dt.days

In [ ]:
# Sequence :: Registers and then Camp Date is Announced=>Goes in a Camp
# Gets the Test Done=>HealthScore=>Camp Ends...

# Registration Date will be the oldest date=>Camp Start Date =>Camp End Date
# Camp Start<First Interaction<Camp end Date = 1 else 0
# Wherever we see the above pattern, we will return 1 else 0.


In [ ]:
def dates_between(start,inter,end):
  if(start<inter<end):
    return 1
  else:
    0

In [ ]:
combined['patient_int_date']=combined.apply(lambda x:dates_between(x['Camp_Start_Date'],x['First_Interaction'],x['Camp_End_Date']),axis=1)

In [ ]:
abs(combined.Camp_Start_Date-combined.Camp_End_Date).dt.days

0          59
1          59
2         751
3          17
4           4
         ... 
110522      9
110523      5
110524      5
110525      5
110526      5
Length: 110527, dtype: int64

In [ ]:
combined.Camp_Start_Date-combined.Camp_End_Date

0         -59 days
1         -59 days
2        -751 days
3         -17 days
4          -4 days
            ...   
110522     -9 days
110523     -5 days
110524     -5 days
110525     -5 days
110526     -5 days
Length: 110527, dtype: timedelta64[ns]

In [ ]:
combined['Registration_Days']=combined.Registration_Date.dt.day
combined['Registration_month']=combined.Registration_Date.dt.month
combined['Registration_years']=combined.Registration_Date.dt.year

In [ ]:
combined['Camp_Start_year']=combined.Camp_Start_Date.dt.year
combined['First_interaction_Year']=combined.First_Interaction.dt.year

In [ ]:
combined["Patients_PerDay"]=combined.groupby("Patient_ID")["Registration_Days"].transform("nunique")

combined["Patients_PerMonth"]=combined.groupby("Patient_ID")["Registration_month"].transform("nunique")

combined["Patients_PerYear"]=combined.groupby("Patient_ID")["Registration_years"].transform("nunique")


In [ ]:
# Patient Count per Health Camp
combined["Patient_PerHC"] = combined.groupby(["Health_Camp_ID"])["Patient_ID"].transform("nunique")


In [ ]:
combined.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Donation,Health_Score,Unnamed: 4,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,social_media,days_interaction,camp_duration,camp_registration,patient_int_date,Registration_Days,Registration_month,Registration_years,Camp_Start_year,First_interaction_Year,Patients_PerDay,Patients_PerMonth,Patients_PerYear,Patient_PerHC
0,489652,6578,2005-09-10,4,0,0,0,2,0,0,0,0,None,None,None,2004-12-06,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2005-08-16,2005-10-14,Third,G,2,0,278.0,59,25.0,NaN,10.0,9.0,2005.0,2005,2004,9,7,3,2837
1,507246,6578,2005-08-18,45,5,0,0,7,0,0,0,0,1,75,40,2004-09-08,C,Others,NaN,NaN,NaN,NaN,NaN,NaN,2005-08-16,2005-10-14,Third,G,2,0,344.0,59,2.0,NaN,18.0,8.0,2005.0,2005,2004,16,12,4,2837
2,523729,6534,2006-04-29,0,0,0,0,0,0,0,0,0,None,None,None,2004-06-22,NaN,NaN,NaN,NaN,NaN,0.402054,NaN,NaN,2005-10-17,2007-11-07,Second,A,2,0,676.0,751,194.0,NaN,29.0,4.0,2006.0,2005,2004,5,4,2,3597
3,524931,6535,2004-02-07,0,0,0,0,0,0,0,0,0,None,None,None,2004-02-07,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004-02-01,2004-02-18,First,E,2,0,0.0,17,6.0,1.0,7.0,2.0,2004.0,2004,2004,4,4,3,1882
4,521364,6529,2006-02-28,15,1,0,0,7,0,0,0,1,1,70,40,2003-07-04,I,Technology,NaN,NaN,NaN,0.845597,NaN,NaN,2006-03-30,2006-04-03,Second,A,2,1,970.0,4,30.0,NaN,28.0,2.0,2006.0,2006,2003,17,9,4,3823


In [ ]:
combined.columns

Index(['Patient_ID', 'Health_Camp_ID', 'Registration_Date', 'Var1', 'Var2',
       'Var3', 'Var4', 'Var5', 'Online_Follower', 'LinkedIn_Shared',
       'Twitter_Shared', 'Facebook_Shared', 'Income', 'Education_Score', 'Age',
       'First_Interaction', 'City_Type', 'Employer_Category', 'Donation',
       'Health_Score', 'Unnamed: 4', 'Health Score', 'Number_of_stall_visited',
       'Last_Stall_Visited_Number', 'Camp_Start_Date', 'Camp_End_Date',
       'Category1', 'Category2', 'Category3', 'social_media',
       'days_interaction', 'camp_duration', 'camp_registration',
       'patient_int_date', 'Registration_Days', 'Registration_month',
       'Registration_years', 'Camp_Start_year', 'First_interaction_Year',
       'Patients_PerDay', 'Patients_PerMonth', 'Patients_PerYear',
       'Patient_PerHC'],
      dtype='object')

In [ ]:
mapped={'First':1,'Second':2,'Third':3}
combined.Category1=combined.Category1.map(mapped)

In [ ]:
combined.Category2=pd.factorize(combined['Category2'])[0]

In [ ]:
combined.City_Type=pd.factorize(combined['City_Type'])[0]

In [ ]:
combined.Employer_Category=pd.factorize(combined['Employer_Category'])[0]

In [ ]:
combined.Donation.fillna(combined.Donation.median(),inplace=True)

In [ ]:
freq=combined.Health_Camp_ID.value_counts(normalize=True)

In [ ]:
combined['health_camp_freq']=combined.Health_Camp_ID.map(freq)

In [ ]:
def outcomes(hs, hs_, stall, last):
    if ((hs>0) | (hs_>0) | (stall>0) | (last>0)):
        return 1
    else:
        return 0


In [ ]:
combined.columns

Index(['Patient_ID', 'Health_Camp_ID', 'Registration_Date', 'Var1', 'Var2',
       'Var3', 'Var4', 'Var5', 'Online_Follower', 'LinkedIn_Shared',
       'Twitter_Shared', 'Facebook_Shared', 'Income', 'Education_Score', 'Age',
       'First_Interaction', 'City_Type', 'Employer_Category', 'Donation',
       'Health_Score', 'Unnamed: 4', 'Health Score', 'Number_of_stall_visited',
       'Last_Stall_Visited_Number', 'Camp_Start_Date', 'Camp_End_Date',
       'Category1', 'Category2', 'Category3', 'social_media',
       'days_interaction', 'camp_duration', 'camp_registration',
       'patient_int_date', 'Registration_Days', 'Registration_month',
       'Registration_years', 'Camp_Start_year', 'First_interaction_Year',
       'Patients_PerDay', 'Patients_PerMonth', 'Patients_PerYear',
       'Patient_PerHC', 'health_camp_freq'],
      dtype='object')

In [ ]:
# generating the target variable


combined['tv']=combined.apply(lambda x:outcomes(x['Health_Score'],x['Health Score'],x['Number_of_stall_visited'],x['Last_Stall_Visited_Number']),axis=1)

In [ ]:
combined.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Donation,Health_Score,Unnamed: 4,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,social_media,days_interaction,camp_duration,camp_registration,patient_int_date,Registration_Days,Registration_month,Registration_years,Camp_Start_year,First_interaction_Year,Patients_PerDay,Patients_PerMonth,Patients_PerYear,Patient_PerHC,health_camp_freq,tv
0,489652,6578,2005-09-10,4,0,0,0,2,0,0,0,0,None,None,None,2004-12-06,-1,-1,30.0,NaN,NaN,NaN,2.0,1.0,2005-08-16,2005-10-14,3,0,2,0,278.0,59,25.0,NaN,10.0,9.0,2005.0,2005,2004,9,7,3,2837,0.025668,1
1,507246,6578,2005-08-18,45,5,0,0,7,0,0,0,0,1,75,40,2004-09-08,0,0,30.0,NaN,NaN,NaN,NaN,NaN,2005-08-16,2005-10-14,3,0,2,0,344.0,59,2.0,NaN,18.0,8.0,2005.0,2005,2004,16,12,4,2837,0.025668,0
2,523729,6534,2006-04-29,0,0,0,0,0,0,0,0,0,None,None,None,2004-06-22,-1,-1,30.0,NaN,NaN,0.402054,NaN,NaN,2005-10-17,2007-11-07,2,1,2,0,676.0,751,194.0,NaN,29.0,4.0,2006.0,2005,2004,5,4,2,3597,0.032544,1
3,524931,6535,2004-02-07,0,0,0,0,0,0,0,0,0,None,None,None,2004-02-07,1,-1,30.0,NaN,NaN,NaN,NaN,NaN,2004-02-01,2004-02-18,1,2,2,0,0.0,17,6.0,1.0,7.0,2.0,2004.0,2004,2004,4,4,3,1882,0.017028,0
4,521364,6529,2006-02-28,15,1,0,0,7,0,0,0,1,1,70,40,2003-07-04,1,1,30.0,NaN,NaN,0.845597,NaN,NaN,2006-03-30,2006-04-03,2,1,2,1,970.0,4,30.0,NaN,28.0,2.0,2006.0,2006,2003,17,9,4,3823,0.034589,1


In [ ]:
combined.tv.value_counts()

0    89993
1    20534
Name: tv, dtype: int64

In [ ]:
# Income, Education Score and Age
combined.Income.replace(to_replace="None", value = np.nan, inplace = True)
combined.Education_Score.replace(to_replace="None", value = np.nan, inplace = True)
combined.Age.replace(to_replace="None", value = np.nan, inplace = True)


In [ ]:
combined.Age=combined.Age.astype(float)

In [ ]:
combined.Education_Score=combined.Education_Score.astype(float)

In [ ]:
combined.groupby(["Health_Camp_ID"])["Patient_ID"].transform("nunique")

0         2837
1         2837
2         3597
3         1882
4         3823
          ... 
110522     166
110523    2180
110524    3041
110525    2441
110526    3041
Name: Patient_ID, Length: 110527, dtype: int64

In [ ]:
combined.groupby("Patient_ID")["Registration_month"].transform("nunique")

0          7
1         12
2          4
3          4
4          9
          ..
110522    11
110523     3
110524     8
110525     3
110526     4
Name: Registration_month, Length: 110527, dtype: int64

In [ ]:
combined.Income.fillna(1,inplace=True)

In [ ]:
new = combined.drop(["Patient_ID", "Health_Camp_ID", "Registration_Date",
              'Online_Follower', 'LinkedIn_Shared',
       'Twitter_Shared', 'Facebook_Shared',
              'First_Interaction', 'Health_Score', 'Unnamed: 4', 'Health Score', 'Number_of_stall_visited',
       'Last_Stall_Visited_Number', 'Camp_Start_Date', 'Camp_End_Date'], axis =1)


In [ ]:
new.shape

(110527, 30)

In [ ]:
new.columns

Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Income', 'Education_Score',
       'Age', 'City_Type', 'Employer_Category', 'Donation', 'Category1',
       'Category2', 'Category3', 'social_media', 'days_interaction',
       'camp_duration', 'camp_registration', 'patient_int_date',
       'Registration_Days', 'Registration_month', 'Registration_years',
       'Camp_Start_year', 'First_interaction_Year', 'Patients_PerDay',
       'Patients_PerMonth', 'Patients_PerYear', 'Patient_PerHC',
       'health_camp_freq', 'tv'],
      dtype='object')

In [ ]:
new.isnull().sum()

Var1                          0
Var2                          0
Var3                          0
Var4                          0
Var5                          0
Income                        0
Education_Score           98167
Age                       77925
City_Type                     0
Employer_Category             0
Donation                      0
Category1                     0
Category2                     0
Category3                     0
social_media                  0
days_interaction            334
camp_duration                 0
camp_registration           334
patient_int_date          82613
Registration_Days           334
Registration_month          334
Registration_years          334
Camp_Start_year               0
First_interaction_Year        0
Patients_PerDay               0
Patients_PerMonth             0
Patients_PerYear              0
Patient_PerHC                 0
health_camp_freq              0
tv                            0
dtype: int64

In [ ]:
new.fillna(-999,inplace=True)

In [ ]:
new.isnull().sum()

Var1                      0
Var2                      0
Var3                      0
Var4                      0
Var5                      0
Income                    0
Education_Score           0
Age                       0
City_Type                 0
Employer_Category         0
Donation                  0
Category1                 0
Category2                 0
Category3                 0
social_media              0
days_interaction          0
camp_duration             0
camp_registration         0
patient_int_date          0
Registration_Days         0
Registration_month        0
Registration_years        0
Camp_Start_year           0
First_interaction_Year    0
Patients_PerDay           0
Patients_PerMonth         0
Patients_PerYear          0
Patient_PerHC             0
health_camp_freq          0
tv                        0
dtype: int64

In [ ]:
new.days_interaction

0         278.0
1         344.0
2         676.0
3           0.0
4         970.0
          ...  
110522    786.0
110523    415.0
110524    736.0
110525    619.0
110526    680.0
Name: days_interaction, Length: 110527, dtype: float64

In [ ]:
# replace null value with -999 to ensure that such category appear differently

new.fillna(-999,inplace=True)

In [ ]:
newtrain=new.loc[0:train.shape[0]-1,]
newtest=new.loc[train.shape[0]:,]

In [ ]:
newtest.shape,test.shape,newtrain.shape,train.shape

((35249, 30), (35249, 8), (75278, 30), (75278, 8))

In [ ]:
X=newtrain.drop('tv',axis=1)
y=newtrain['tv']
xtest=newtest.drop('tv',axis=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
pred_rf=rf.fit(X,y).predict_proba(xtest)

In [ ]:
# Gaussian NB
# from sklearn.naive_bayes import GaussianNB
# gnb = GaussianNB()

# pred_gnb = gnb.fit(X, y).predict_proba(newtest)

solution = pd.DataFrame({"Patient_ID":test.Patient_ID,
                        "Health_Camp_ID":test.Health_Camp_ID,
                        "Outcome": pred_rf[:, 1]}) 

solution.to_csv("rfMODELhc.csv", index = False) # 0.65


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc=GradientBoostingClassifier()
pred_gbc=rf.fit(X,y).predict_proba(xtest)

In [ ]:
solution = pd.DataFrame({"Patient_ID":test.Patient_ID,
                        "Health_Camp_ID":test.Health_Camp_ID,
                        "Outcome": pred_gbc[:, 1]}) 

solution.to_csv("gbcMODELhc.csv", index = False)

In [ ]:
model1=gbc.fit(X,y)

In [ ]:
df=DataFrame({'v':model1.feature_importances_,'fea':X.columns}).sort_values('v',ascending=False).set_index('fea')

In [ ]:
fea=df[df['v']>0.001].index

In [ ]:
newx=X[fea]
newtest=xtest[fea]

In [ ]:
len(model1.feature_names_in_)

29

In [ ]:
# Cross Validation
n = 10
gbm = GradientBoostingClassifier()
pred_model = pd.DataFrame()

for i in range(n):
    folds = next(kfold.split(newX)) # Splitting the data in even groups...
    xtrain = 
gbm_cv = pred_model.mean(axis=1)

solution = pd.DataFrame({"Patient_ID":test.Patient_ID,
                        "Health_Camp_ID":test.Health_Camp_ID,
                        "Outcome": gbm_cv}) 

solution.to_csv("GBM5CVhc.csv", index = False) # 0.745
